In [222]:
import pandas as pd
from time import time

start_time = time()
print pd.read_csv('only_buses/2016-02-01-EZ.csv')
print time() - start_time

              4     5   7      8        9          10        11          12  \
0        170     8207   1  46069  45069.0  2016-02-01  04:54:58  2016-02-01   
1        170     8207   1  46069  41069.0  2016-02-01  04:54:58  2016-02-01   
2        170     8207   1  46069  42119.0  2016-02-01  04:54:59  2016-02-01   
3        170     8207   1  46069  45139.0  2016-02-01  04:54:59  2016-02-01   
4        170     8207   1  46069  45139.0  2016-02-01  04:55:01  2016-02-01   
5        170     8207   1  46069  42059.0  2016-02-01  04:55:01  2016-02-01   
6        170     8207   1  46069  45139.0  2016-02-01  04:55:02  2016-02-01   
7        170     8207   1  46069  42089.0  2016-02-01  04:55:03  2016-02-01   
8        170     8207   1  46069  42119.0  2016-02-01  04:55:03  2016-02-01   
9        170     8207   1  46069  41029.0  2016-02-01  04:55:05  2016-02-01   
10       170     8207   1  46069  45139.0  2016-02-01  04:55:06  2016-02-01   
11       170     8207   1  46069  45139.0  2016-02-0

In [15]:
pd.Timedelta(seconds=2066), pd.Timedelta(seconds=1222), pd.Timedelta(seconds=1561), pd.Timedelta(seconds=1118), pd.Timedelta(seconds=2064) 
# 2066.461450 seconds 01-02
# 1222.099073         07-02           
# 1561.563673         06-02
# 1118.694052         08-02
# 2064.430148         02-02
# 2064.430148         05-02

(Timedelta('0 days 00:34:26'),
 Timedelta('0 days 00:20:22'),
 Timedelta('0 days 00:26:01'),
 Timedelta('0 days 00:18:38'),
 Timedelta('0 days 00:34:24'))

In [84]:
bus_transactions = pd.read_csv('test_splitted/bus2_2016-02-01_new')

bus_route = pd.read_csv('BusRoute/2bus.csv', header=None)
bus_route.drop(bus_route.columns[[0, 3, 4, 5, 6, 9]], axis = 1, inplace = True)
bus_route.columns = ['station', 'distance', 'stops_order', 'bus_direction']
grouped_bus_route = bus_route.groupby(['bus_direction'])

In [247]:
mozhno = True

def determine_direction(stops, grouped_bus_route):
    global mozhno
    stops['station'] = stops['station'].apply(int)
    groups = grouped_bus_route.groups
    result = {}
    answer = None
    minimal_shape = None
    for direction in groups:
        df = grouped_bus_route.get_group(direction)
        df['station'] = df['station'].apply(int)
        df = df.join(stops.set_index('station'), on='station', how='left', lsuffix='_caller', rsuffix='_other')
#         result[direction] = df
        notnull_df_shape = df[pd.notnull(df['bus_registration_number'])].shape
        if answer is not None and notnull_df_shape > minimal_shape or answer is None:
            minimal_shape = notnull_df_shape
            answer = df
        
    return answer

In [134]:
get_group = grouped_bus_transactions.get_group
[(group_name, get_group(group_name)['station']) for group_name in grouped_bus_transactions.groups]

[((3828, 12), 7982    10017.0
  7999     5013.0
  8028     5023.0
  8064     4142.0
  8073     1012.0
  8087     1113.0
  8091     1121.0
  8093     1211.0
  8107    80011.0
  8115    80031.0
  8125    80071.0
  8137    80091.0
  8144    81031.0
  8156    81051.0
  8165    82032.0
  8175    82061.0
  8184    83011.0
  8185    83062.0
  8202    84011.0
  8206    84021.0
  8221    84031.0
  8229    84041.0
  8235    84061.0
  8248    85091.0
  8275    85071.0
  8287    96091.0
  8313    97011.0
  8316    97031.0
  8318    97041.0
  8321    97051.0
  8345    98071.0
  8354    99139.0
  8356    99021.0
  Name: station, dtype: float64), ((3514, 1), 18     99009.0
  22     99131.0
  37     97099.0
  49     97049.0
  53     97039.0
  66     96069.0
  74     96059.0
  89     96089.0
  98     85089.0
  108    85069.0
  121    85099.0
  129    84069.0
  134    84059.0
  149    84049.0
  162    84039.0
  177    84029.0
  189    84019.0
  206    83099.0
  227    83059.0
  233    83049.0
  246    8

In [248]:
grouped_bus_transactions = bus_transactions.groupby(['bus_registration_number', 'bus_trip_number'])
get_group = grouped_bus_transactions.get_group
# print grouped_bus_transactions.get_group((2,1))
# grouped_stops = [(group_name, get_group(group_name)) for group_name in grouped_bus_transactions.groups]

grouped_stops = [(group_name, determine_direction(get_group(group_name), grouped_bus_route)) for group_name in grouped_bus_transactions.groups]

grouped_stops[2]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


((8301, 3),
      station  distance  stops_order  bus_direction  bus_registration_number  \
 66     10589       0.0            1              2                   8301.0   
 67     10017       0.2            2              2                   8301.0   
 68      5013       1.2            3              2                   8301.0   
 69      5023       1.6            4              2                      NaN   
 70      4222       1.9            5              2                   8301.0   
 71      4142       2.4            6              2                      NaN   
 72      1012       2.9            7              2                   8301.0   
 73      1113       3.5            8              2                   8301.0   
 74      1121       3.9            9              2                   8301.0   
 75      1211       4.2           10              2                      NaN   
 76      1311       4.5           11              2                   8301.0   
 77     80011       5.2     

In [187]:
info, stops = grouped_stops[0]
first
# determine_direction(first, grouped_bus_route)
stops['station'] = stops['station'].apply(int)
stops['num'] = 611
direction = 2
df = grouped_bus_route.get_group(direction)
print stops.columns.values
print df.columns.values
df = df.join(stops.set_index('station'), on='station', how='left', lsuffix='_caller', rsuffix='_other')
print stops
print '==' * 30
print df

['bus_registration_number' 'bus_trip_number' 'station' 'median' 'datetime'
 'num']
['station' 'distance' 'stops_order' 'bus_direction']
      bus_registration_number  bus_trip_number  station   median  \
7982                     3828               12    10017  82513.5   
7999                     3828               12     5013  82705.5   
8028                     3828               12     5023  82938.5   
8064                     3828               12     4142  83303.0   
8073                     3828               12     1012  83405.0   
8087                     3828               12     1113  83565.0   
8091                     3828               12     1121  83593.0   
8093                     3828               12     1211  83626.0   
8107                     3828               12    80011  83785.0   
8115                     3828               12    80031  83897.5   
8125                     3828               12    80071  83994.5   
8137                     3828               12  

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [240]:
group_name = (8301, 3)
determine_direction(get_group(group_name), grouped_bus_route)

direction: 1
,     station  distance  stops_order  bus_direction  bus_registration_number  \
0     99009       0.0            1              1                   8301.0   
65    10589      25.3           66              1                   8301.0   

    bus_trip_number   median                 datetime  
0               3.0  41712.0  2016-02-01 11:35:12.000  
65              3.0  36653.5  2016-02-01 10:10:53.500  
direction: 2
,      station  distance  stops_order  bus_direction  bus_registration_number  \
66     10589       0.0            1              2                   8301.0   
67     10017       0.2            2              2                   8301.0   
68      5013       1.2            3              2                   8301.0   
70      4222       1.9            5              2                   8301.0   
72      1012       2.9            7              2                   8301.0   
73      1113       3.5            8              2                   8301.0   
74      1121  

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


(    station  distance  stops_order  bus_direction  bus_registration_number  \
 0     99009       0.0            1              1                   8301.0   
 1     99131       0.2            2              1                      NaN   
 2     99049       0.5            3              1                      NaN   
 3     99039       0.9            4              1                      NaN   
 4     99029       1.2            5              1                      NaN   
 5     99019       1.6            6              1                      NaN   
 6     98079       2.2            7              1                      NaN   
 7     98069       2.6            8              1                      NaN   
 8     97099       3.0            9              1                      NaN   
 9     97089       3.5           10              1                      NaN   
 10    97079       3.9           11              1                      NaN   
 11    97069       4.3           12              1  

In [265]:
left_joint = grouped_stops[0][1]
left_joint[pd.notnull(left_joint['datetime'])]


,station,distance,stops_order,bus_direction,bus_registration_number,bus_trip_number,median,datetime
67,10017,0.2,2,2,3828.0,12.0,82513.5,2016-02-01 22:55:13.500
68,5013,1.2,3,2,3828.0,12.0,82705.5,2016-02-01 22:58:25.500
69,5023,1.6,4,2,3828.0,12.0,82938.5,2016-02-01 23:02:18.500
71,4142,2.4,6,2,3828.0,12.0,83303.0,2016-02-01 23:08:23.000
72,1012,2.9,7,2,3828.0,12.0,83405.0,2016-02-01 23:10:05.000
73,1113,3.5,8,2,3828.0,12.0,83565.0,2016-02-01 23:12:45.000
74,1121,3.9,9,2,3828.0,12.0,83593.0,2016-02-01 23:13:13.000
75,1211,4.2,10,2,3828.0,12.0,83626.0,2016-02-01 23:13:46.000
77,80011,5.2,12,2,3828.0,12.0,83785.0,2016-02-01 23:16:25.000
78,80031,5.7,13,2,3828.0,12.0,83897.5,2016-02-01 23:18:17.500
